# SGAN training

In [1]:
import os, tqdm
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from   torch.utils.data import TensorDataset, DataLoader
from   torch.utils.tensorboard import SummaryWriter


from scripts.Evaluator     import evaluator
from scripts.Generator     import generatorNet
from scripts.Discriminator import ensembleNet


import time

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device    = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('the device is %s' % device)

the device is cuda


## Data preprocess

In this step, the input features will be generated. In detail:
1. labeled training data consists of 500 oncogenic variants and 500 benign variants;
2. others labeled variants are used as validation set;
3. 60,000 unlabeled variants are used for unsupervised training;

In [3]:
shuffle_seed           = 37
oncogenic_variant_size = 1000
benign_variant_size    = 1000
batchsize              = 2000

In [4]:
training_unlabeled = pd.read_csv("example/training_data/unlabeled_training.dat")
training_labeled   = pd.read_csv("example/training_data/labeled_training.dat")
training_labeled.head(2)

,SIFT_score,Polyphen2_HDIV_score,Polyphen2_HVAR_score,LRT_score,MutationTaster_score,MutationAssessor_score,FATHMM_score,PROVEAN_score,VEST3_score,CADD_raw,...,evs_10_2,evs_11_-1,evs_11_0,evs_11_1,evs_11_2,evs_12_-1,evs_12_0,evs_12_1,evs_12_2,true_label
0,0.012,0.996,0.877,0.000,0.967,0.631809,0.617818,0.497366,0.860861,0.426250,...,0.070834,0.098091,0.938011,0.073036,0.070834,0.098091,0.938011,0.073036,0.070834,0
1,0.003,0.790,0.365,0.694,1.000,0.491245,0.672773,0.542581,0.067067,0.442278,...,0.077686,0.106276,0.091509,0.916682,0.077686,0.106276,0.091509,0.916682,0.077686,0


In [5]:
## labeled
training_labeled = training_labeled.values

np.random.seed(shuffle_seed)
np.random.shuffle(training_labeled)

labeled_features, labeled_targets = training_labeled[:, :-1], training_labeled[:, -1]

training_features   = np.vstack( (labeled_features[labeled_targets == 1][:oncogenic_variant_size], 
                                  labeled_features[labeled_targets == 0][:benign_variant_size]) )

validation_features = np.vstack( (labeled_features[labeled_targets == 1][oncogenic_variant_size:], 
                                  labeled_features[labeled_targets == 0][benign_variant_size:]) )


training_targets    = np.hstack( (labeled_targets[labeled_targets == 1][:oncogenic_variant_size], 
                                  labeled_targets[labeled_targets == 0][:benign_variant_size]) )

validation_targets  = np.hstack( (labeled_targets[labeled_targets == 1][oncogenic_variant_size:], 
                                  labeled_targets[labeled_targets == 0][benign_variant_size:]) )


## unlabeled
unlabeled_features = training_unlabeled.values


In [6]:
tensor_dat       = TensorDataset(torch.Tensor(training_features[:, np.newaxis, :]), torch.Tensor(training_targets).long())
training_batch   = DataLoader(dataset=tensor_dat, batch_size = batchsize, shuffle=True)

tensor_dat       = TensorDataset(torch.Tensor(validation_features[:, np.newaxis, :]), torch.Tensor(validation_targets).long())
validation_batch = DataLoader(dataset=tensor_dat, batch_size = batchsize, shuffle=False)

tensor_dat       = TensorDataset(torch.Tensor(unlabeled_features[:, np.newaxis, :]))
unlabeled_batch  = DataLoader(dataset=tensor_dat, batch_size = batchsize, shuffle=False)

## Model settings
- In this step, we used `CrossEntropyLoss` to calculate the loss of supervised learning process. The loss of unsupervised learning are shown in training loop.
- We used `LambdaLR` to update learning rate in each epoch. 
- The scripts of disciminator/generator model are stored in `scripts/`

In [7]:
discrminator  = ensembleNet().to(device)
generator     = generatorNet().to(device)
cross_entropy = nn.CrossEntropyLoss()
optimizerDis  = optim.AdamW(discrminator.parameters(), lr = 0.01)
optimizerGen  = optim.AdamW(generator.parameters(), lr = 0.01)

schedulerDis  = optim.lr_scheduler.LambdaLR(optimizerDis, lambda epoch: 0.9**epoch)
schedulerGen  = optim.lr_scheduler.LambdaLR(optimizerGen, lambda epoch: 0.9**epoch)

## Model training 

- `tensorboard.SummaryWritter` is used for record the evaluation mertics in each training step;
- `num_epochs` is the number of training epoch;
- users can save the model stats using `torch.save(discrminator.state_dict(), './saves/myTrain.pt')`

In [8]:
data_fix = time.strftime('%Y-%m-%d_%H-%M')

writer = SummaryWriter(comment=data_fix)


if not os.path.exists( "saves/%s" % data_fix ):
    os.mkdir( "saves/%s" % data_fix )

num_epochs  = 500
global_step = 1
iter_labeled = iter(training_batch)


for epoch in tqdm.tqdm(range(num_epochs)):
    for step, x_unlabeled in enumerate(unlabeled_batch):

        #################################################################################  Classifier/Discriminator 
        discrminator.train()
        generator.eval()
        
        optimizerDis.zero_grad()
        
        ## label
        try:
            x_labeled, y_labeled = next(iter_labeled)
            x_labeled, y_labeled = x_labeled.to(device), y_labeled.to(device)
        except StopIteration:
            iter_labeled = iter(training_batch)
            x_labeled, y_labeled = next(iter_labeled)
            x_labeled, y_labeled = x_labeled.to(device), y_labeled.to(device)
        
        
        _, outClassLabeled  = discrminator(x_labeled)
        lossLabeled      = cross_entropy(outClassLabeled, y_labeled)
        
        
        ## unlabel
        x_unlabeled = x_unlabeled[0].to(device)
        _, outClassUnlabeled  = discrminator(x_unlabeled)
        
        logz_unlabeled = torch.logsumexp(outClassUnlabeled, dim=1)
        lossUnlabeled  = -0.5 * torch.mean(logz_unlabeled) + 0.5 * torch.mean(F.softplus(logz_unlabeled))
        
        ## Fake
        fakeNoise1       = torch.randn(x_unlabeled.size(0), 30, device=device)
        x_Fake1          = ( generator(fakeNoise1) + 1.0 ) / 2
        _, outClassFake1 = discrminator(x_Fake1)

        logz_fake1 = torch.logsumexp(outClassFake1, dim=1)
        lossFake  = 0.5 * torch.mean(F.softplus(logz_fake1))
        

        ## loss
        totalLoss = lossLabeled + lossUnlabeled + lossFake
        
        ## optimization
        writer.add_scalar("training_loss/supervised", lossLabeled, global_step)
        writer.add_scalar("training_loss/unsupervised", lossUnlabeled+lossFake, global_step)
        writer.add_scalar("training_loss/Discriminator", totalLoss, global_step)

        totalLoss.backward()
        optimizerDis.step()
        
                
        #################################################################################  Generator
        discrminator.eval()
        generator.train()
        optimizerGen.zero_grad()
        
        fakeNoise2 = torch.randn(x_unlabeled.size(0), 30, device=device)
        x_Fake2    = ( generator(fakeNoise2) + 1.0 ) / 2
        
        ## loss
        y_pred_unlabeled, _ = discrminator(x_unlabeled)
        y_pred_fake, _      = discrminator(x_Fake2)
        mom_real = torch.mean(y_pred_unlabeled, dim=0)
        mom_fake = torch.mean(y_pred_fake, dim=0)
        diff = mom_fake * 100 - mom_real * 100
        lossG = torch.mean(diff * diff)
        
        
        ## optimization
        writer.add_scalar("training_loss/Generator", lossG, global_step)
        lossG.backward()        
        optimizerGen.step()

        global_step += 1

    
    torch.save(discrminator.state_dict(), 'saves/%s/discrminator_epoch%d.pt' % ( data_fix, epoch + 1 ))
    torch.save(generator.state_dict(),    'saves/%s/generator_epoch%d.pt' % ( data_fix, epoch + 1 ))
    
    
    training_loss, training_accuracy = evaluator(discrminator, cross_entropy, device, training_batch, False)
    writer.add_scalar("accuracy/training",  training_accuracy, epoch)
    
    validation_loss, validation_accuracy = evaluator(discrminator, cross_entropy, device, validation_batch, False)
    writer.add_scalar("accuracy/validation", validation_accuracy, epoch)
    writer.flush()

writer.close()

100%|██████████| 500/500 [18:59<00:00,  2.28s/it]


## Model performance on testing data
- we provided a testing set consisting of 6,226 variants with reliable labels;
- users can use this testing set to evaluate your trained model

In [9]:
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.metrics import f1_score 
import math

def myEval(model, device, test_loader, display = False):
    model.eval()
    
    target_list = []
    output_list = []
    
    with torch.no_grad():
        for testdata in test_loader:
            data, target = testdata
            data, target = data.to(device), target.to(device)
            _, output = model(data)
            softmax2_score = [ math.exp(i[1]) / ( math.exp(i[0]) + math.exp(i[1]) ) for i in output.cpu().numpy() ]
            target_list += target.cpu().tolist()
            output_list += softmax2_score

    return target_list, output_list


def evaluation_df(pred_score, labeled_y):
    def TP_table(pred_score, labeled_y, threshold):
        y_pred = [0 if i < threshold else 1 for i in pred_score]
        y_true = list(labeled_y)


        fpr, tpr, _ = roc_curve(y_true, pred_score)
        auc_val = auc(fpr, tpr)


        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        # TP TN FP FN sensitivity specificity Accuracy
        sensitivity = tp/(tp+fn)

        specificity = tn/(tn+fp)
        accuracy    = (tp+tn)/(tp+tn+fp+fn)
        
        F1 = f1_score(y_true, y_pred)
        
        try:
            MCC = ((tp*tn)-(fp*fn)) / ((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))**0.5
        except:
            MCC = np.nan

        return [threshold, tp, fp, tn, fn, sensitivity, specificity, accuracy, auc_val, MCC, F1]

    res = []
    for i in range(1,20):
        threshold = i / 20
        res.append(TP_table(pred_score, labeled_y, threshold))

    res = pd.DataFrame(res, columns=['threshold', 'TP', 'FP', 'TN', 'FN', 'sen', 'spe', 'Acc', 'AUC', 'MCC', 'F1'])
    return res


In [10]:
testing_labeled = pd.read_csv("example/training_data/labeled_testing.dat").values
labeled_features, labeled_targets = testing_labeled[:, :-1], testing_labeled[:, -1]


tensor_dat = TensorDataset(torch.Tensor(labeled_features[:, np.newaxis, :]), torch.Tensor(labeled_targets).long())
testing_batch = DataLoader(dataset=tensor_dat, batch_size = batchsize, shuffle=False)

label, pred = myEval(discrminator, device, testing_batch)
evaluation_df(pred, label)

,threshold,TP,FP,TN,FN,sen,spe,Acc,AUC,MCC,F1
0,0.05,1280,3751,1140,55,0.958801,0.233081,0.388693,0.822681,0.199970,0.402136
1,0.10,1257,3357,1534,78,0.941573,0.313637,0.448281,0.822681,0.239120,0.422592
2,0.15,1239,3124,1767,96,0.928090,0.361276,0.482814,0.822681,0.259351,0.434889
3,0.20,1222,2958,1933,113,0.915356,0.395216,0.506746,0.822681,0.271369,0.443155
4,0.25,1207,2835,2056,128,0.904120,0.420364,0.524093,0.822681,0.279067,0.448949
5,0.30,1199,2699,2192,136,0.898127,0.448170,0.544651,0.822681,0.293749,0.458246
6,0.35,1193,2606,2285,142,0.893633,0.467185,0.558625,0.822681,0.303639,0.464745
7,0.40,1187,2514,2377,148,0.889139,0.485995,0.572438,0.822681,0.313570,0.471406
8,0.45,1175,2411,2480,160,0.880150,0.507054,0.587054,0.822681,0.321567,0.477545
9,0.50,1162,2291,2600,173,0.870412,0.531589,0.604240,0.822681,0.331966,0.485380
